### LX 496/796  Formalizing meaning

In this third homework, you will become familiar with creating context free grammars in NLTK.  Also, parsing, drawing, and traversing syntactic trees.  And we will get started with a little bit of the work on SHRDLU.

## Longer term goal: A tiny version of SHRDLU

A quick word about SHRDLU.  [SHRDLU](https://en.wikipedia.org/wiki/SHRDLU) is Terry Winograd's program from 1971 that could converse about things in its very limited universe of a block world.  You can find a video of it doing its thing [on YouTube here](https://www.youtube.com/watch?v=bo4RvYJYOzI).  You can see it doing a lot of pretty interesting things.  Learning names for things, remembering what it was talking about, moving blocks around, making plans to accomplish goals.  Ours will not be that sophisticated.

In service of this goal, we will need to accept input, parse the trees, and assign a semantics to the parsed sentences so that we can query or act on the blocks world. So there are a couple of pre-requisites.  In this first assignment (of two) we will work on doing the semantic parsing and world representation, and some of the display planning.  The second assignment will get deeper into making it actual work.

## Formalizing meaning ##

This largely follows the discussion from chapter 10 in the NLTK book, but I will try to elaborate
on it here somewhat.
Also, we went through much of this first part in class.

However, still do the tasks, even if you kind of "got the answers" in class.
This will contain a bit more by way of exercises, to help make it clearer what the concepts are
here.  We will start by creating a little world that we can evaluate sentences against.

In this world, there are four people: Andrea, Bobby, Chris, and Dana.  These are our *individuals*.
An "individual" need not be a person, it's just some kind of entity that we can refer to.
So, let's also add a couple of non-human individuals as well.  To keep things somewhat simple, they
will be the Moon and the Sun.  We are going to pretend their names are `the_sun` and `the_moon`
however (having a space in there makes things not work, so we will use `_` instead of a space).

So, step one, let's define a set of our individuals.  (There is no intrinsic order to these
individuals, they are just the individuals in our model of the world, so it should be a set and
not a list.)

In [ ]:
# define our individuals
dom = {'a', 'b', 'c', 'd', 'm', 's'}

Now we will build up some information about how English words are mapped to these individuals.
First off, we will set up the names.
What we do is
set up a multi-line string first, using then `"""` delimiters on each end.

In [ ]:
# define the mapping from english words => individuals
names = """
andrea => a
bobby => b
chris => c
dana => d
the_sun => s
the_moon => m
"""

As we did in class, and as it is done in the textbook, we
will use NLTK's `fromstring` function to create a `Valuation` from these.

> The `Valuation` is the name of the mapping from English to model objects

In [ ]:
import nltk
# create the Valuation from our names above
val = nltk.Valuation.fromstring(names)
# make sure that it worked by printing the Valuation back out again
print(val)

In [ ]:
assert False, "Go step by step first. Later, you can comment this out."

Now that we have done this, we can "evaluate" the English words and get their referents.

In [ ]:
print(val['bobby'])
print(val['the_moon'])

This tells us which individual in our set of individuals is being referred to by "bobby" and by "the moon".
Doing this counts, at least in a certain sense, as translating from English into "semantics."  We are
determining the meaning of the word "bobby," for example.

The individuals in this world have properties and relationships, however, as well.  For example, some of the
individuals are people.  So, we define "person" as being something that holds of the individuals
a, b, c, and d.  For the moment, we are going to create a new Valuation to hold this information,
and we will merge these together shortly.

In [ ]:
valp = nltk.Valuation.fromstring("person => {a, b, c, d}")
print(valp)

It's kind of a pain to keep typing this `nltk.Valuation.fromstring` thing, let's give it a shorter
name.  I'm going with `vfs` for "valuation-from-string":

In [ ]:
vfs = nltk.Valuation.fromstring

Now, the sun and the moon are not people, what are they?

In [ ]:
valsb = vfs("spaceball => {s, m}")

So far, we have three different Valuations (`val`, `valp`, and `valsb`).  Just to be sure, we will print them out.

In [ ]:
print(valp)
print(valsb)

But we need to merge
them together into one.  It is possible to combine two Valuations using `update`.  So, let's
add `valp` and `valsb` to `val`. First, we'll add the spaceballs and then see what the effect was.

In [ ]:
val.update(valsb)
print(val)

Good, that's what we wanted, now let's add the people.

In [ ]:
val.update(valp)
print(val)

>The `update` function is actually fairly general.  It is defined for Valuations, but it is
>also defined for just regular sets, as well as for dictionaries.  If you call `update` on a
>set or a dictionary, it merges the argument of `update` into it (with priority given to the
>additions, if there is a conflict).

>```python
>aset = {1, 2, 3}
>aset.update({3, 4, 5})
>print(aset) # => {1, 2, 3, 4, 5}
>adict = {'a': 1, 'b': 2}
>adict.update({'b': 4, 'c': 6})
>print(adict) # => {'a': 1, 'b': 4, 'c': 6}
>```

Now, all of our world-building work to date is represented in `val`.  Let's do a little bit more building.
Let us suppose that Andrea and Bobby are from Boston, while Chris and Dana are from Cambridge.

In [ ]:
val.update(vfs("bostonian => {a, b}"))
val.update(vfs("cantabrigian => {c, d}"))

Now, we've defined the mapping between names and individuals, and we've defined some nouns/predicates
that hold of sets of individuals.  What remains is to define some relationships between them.
Relationships are asymmetrical, so just because Andrea likes Bobby does not mean that Bobby likes Andrea.
But let's start with that.

In [ ]:
val.update(vfs("likes => {(a, b)}"))

Ok, now let's (attempt to) make it mutual.  This isn't going to work, but it seems like it might have.  We're trying to add "b likes a" (Bobby likes Andrea) to `val`, and we know that `update` can add things to `val`.  But when we try it, we don't get the mutual liking we had hoped to get.

In [ ]:
val.update(vfs("likes => {(b, a)}"))
print(val['likes'])

Hmm.  That didn't really work.  Instead of making Andrea and Bobby like each other, Bobby started
liking Andrea and Andrea stopped liking Bobby.  This simply replaced the liking pair, rather than
adding to it.  It appears that `update` only operates on the level of the word, and it will replace the specification of the word if the word is already there.  So, to get the mutual liking, we could spell it out fully like this:

In [ ]:
val.update(vfs("likes => {(b, a), (a, b)}"))
print(val['likes'])

And that gets what we want.  But it would be nice to be able to add relations in stages, rather
than redefine the relation in full every time.

It is possible to treat `val['likes']` as the kind of thing you can `update`.  That is, you don't need to update `val` as a whole, you can target a single predicate and update it.  Like:

> `val['likes'].update(...)`

But then we face the question of what form the argument should take.  Judging by what was printed above, it looks like we might be able to update with `{('b', 'a')}`.  Let's try.

In [ ]:
# reset it to just Andrea likes Bobby and see if we can add the reverse
val.update(vfs("likes => {(a, b)}"))
print(val['likes'])
# now try to add the reverse
val['likes'].update({('b', 'a')})
print(val['likes'])

Ok, that appears to have worked.  So, if you want to add a particular relationship or individual to a predicate, you can address the predicate and `update` it.

Now, let's finish setting up the world (with respect to likings).  Here is the goal state:

- Andrea likes everyone
- Everyone likes Dana
- Bobby likes Andrea
- Dana likes Chris
- Andrea and Bobby like the Sun
- Chris and Dana like the Moon

To get you started, we'll do "Andrea likes everyone" because it also allows me to remind you that for something like this we can actually do this somewhat programmatically.  We know who all the people are, they're the people referred to in `val['person']`.

In [ ]:
# go through everyone "person" refers to, add them to Andrea's "like" list:
for (x,) in val['person']:
    val['likes'].update({('a', x)})
# display the result to see if it worked
print(val['likes'])

## TASK 1

Finish setting up `val['likes']` to represent the world situation described above.


In [ ]:
# Answer 1: Finish setting up val['likes']



If it worked, the following command will succeed (will print no error).  If it stops with an error, then you haven't quite gotten the definition of `val['likes']` right yet.

> If you are comfortable enough with Python to see how this verification below works, then you will also see how you might be able to set it up that way, using `set`, `zip`, and strings.  But that's not important or expected, this was intended to be an opaque test just to make sure you got the right result.

In [ ]:
assert val['likes'] == set(zip("aaaaabcdbdabcd", "abcdddddacssmm")), \
    "World is not set up properly yet."

At this point, your valuation function (`val`) will have passed the test for being set up, but let's do a more transparent test on this world.  We know it is supposed to be the case that Andrea likes everyone (that is, all the people).  How can we verify that part?  (Short of just looking at `print(val['likes'])` and checking by hand whether all the expected pairs are there.)

If you type the following, you should get `True` if Andrea likes all the people.

In [ ]:
not False in [(val['andrea'], x) in val['likes'] for (x,) in val['person']]

Got `True`? Great. But why? If you just blindly executed the cell and got `True` without figuring
out what it is doing, that's fine.  But now we're going to figure out what it is doing.

First of all, remind yourself what `val['andrea']`, `val['person']`, and `val['likes']` are:

In [ ]:
print(val['andrea'])
print(val['person'])
print(val['likes'])

We are trying to determine whether Andrea likes all the people.  So, we check, for each person,
whether it is true that Andrea likes that person.  When we're done checking people, we should not
have found any that yield `False`.  As you just saw, `val['person']` is a set of 1-tuples, like `('a',)`.
So to go through the people, we want to use `for (x,) in val['person']` in order to set `x` to be
the individual in our domain that corresponds to the person (e.g., `'a'`).  To determine whether
Andrea likes the person in `x`, we need to find out whether the pair that has Andrea as the first
member and `x` as the second member is in the set of "likings" in `val['likes']`.  The individual
that Andrea represents is `val['andrea']` (which will be `'a'`).  So, we evaluate whether the
pair `(val['andrea'], x)` is in `val['likes']`.  The expression `(val['andrea'], x) in val['likes']`
will be `True` if Andrea likes `x` and `False` otherwise.  The list that this list comprehension
builds will be a list of `True` or `False` values (one for each person).  If Andrea indeed likes every person, then the
list should be `[True, True, True, True]`.  Finally, we check to see if `False` is anywhere in that
list.  If it is, we failed: Andrea doesn't like every person.  If there is no `False` in there,
then we succeeded.  So, `not False in [...]` is `True` if we succeeded.

## TASK 2

Use the same technique to verify that every person likes Dana.

In [ ]:
# Answer 2: Verify that every person likes Dana



Now, let's formalize our model of the world into an official NLTK model.  A model is just a
pairing of a domain and a Valuation function.

In [ ]:
m = nltk.Model(dom, val)

Once we have a model defined, we can use the model's `evaluate` function to test the truth
of things in the model.  In order to use `evaluate` we also need to set up an "assignment function"
(which can be thought of as a record of who we're pointing to).  To begin with, we'll just set up
an empty assignment function (we aren't pointing at anything).

In [ ]:
g = nltk.Assignment(dom)

Now, we can verify that Dana likes Chris, and verify that Bobby does not like Chris, like so:

In [ ]:
print(m.evaluate('likes(dana, chris)', g))
print(m.evaluate('-likes(bobby, chris)', g))

## TASK 3

Use `evaluate` to verify that Dana does not like Bobby, and that Chris likes the Moon.

In [ ]:
# Answer 3: Verify that Dana does not like Bobby, that Chris likes the Moon



We can also use quantifiers like `all` and `exists` with `evaluate`.  For example, we can re-verify
that Andrea likes every person, like so:

In [ ]:
print(m.evaluate('all x.(person(x) -> likes(andrea, x))', g))

The way this works is pretty much exactly how our home-spun version from Task 2 worked.  It goes through
all of the individuals in the domain one by one, and for each it checks to see if it's a person,
and if it is a person, then it checks to see if it is the second member of a pair, whose first member
is Andrea, that can be found in the list of "likings".

## TASK 4

Use `evaluate` to verify that everybody likes Dana.

In [ ]:
# Answer 4: Verify that everybody likes Dana (using "all x")



You can also use `exists`, which is true if the condition is met for at least one of the individuals in the
domain.  So, if we want to ascertain that at least *somebody* likes Bobby, we can do the following:

In [ ]:
print(m.evaluate('exists x.(person(x) & likes(x, bobby))', g))

What that means is that we can find *some* `x` in our domain `dom` such that `x` is both a `person` and
in a `likes` relation with Bobby.

## TASK 5

Use `evaluate` to verify that every Bostonian likes the Sun, and that no spaceballs are from Cambridge.

In [ ]:
# Answer 5: Verify that every Bostonian likes the Sun, no spaceballs are from Cambridge



The string that we give to `evaluate` is first interpreted as a "semantic Expression"
built from a string.  If we don't want to evaluate immediately, we can define such expressions
directly.  The function that does this is `nltk.sem.Expression.fromstring`.  Like before, we'll
give it a shorter name (`sfs`) to save on some typing.  Then we'll define a formula `f1` to be
"x likes the Moon".

In [ ]:
sfs = nltk.sem.Expression.fromstring
f1 = sfs('likes(x, the_moon)')
print(f1)

So, is "x likes the Moon" true?  No idea.  We can't decide that until we know who `x` is
supposed to be.  Once we know who `x` is, then we can figure out whether it's true.  Because
we don't know who `x` is, `x` is considered a "free variable."  Although it's kind of obvious,
we can interrogate `f1` to ask it what its free variables are:

In [ ]:
print(f1.free())

If we want to know who likes the Moon, we can ask the model to tell us which individuals,
when substituted in for `x`, would make `f1` true:

In [ ]:
print(m.satisfiers(f1, 'x', g))

## TASK 6

Use `satsifiers` to determine who/what Chris likes.

In [ ]:
# Answer 6: Use satisfiers to determine who/what Chris likes



**Answer 6** (markdown). Who/what does Chris like, based on the value returned just above?

One way that we can set a value for `x` is to use `x` to point to an individual.  That is,
suppose we point (with our "x" finger) at Bobby, and then ask whether "x likes the Moon"
is true.  Since this tells us who `x` is (namely, Bobby), we can decide whether "x likes the Moon"
is true.  It's true if (and only if) Bobby likes the Moon.

In [ ]:
# reminder that f1 is the expression x likes the moon
print(f1)
# point to Bobby with x
g['x'] = 'b'
# does x like the moon, given that x is Bobby?
print(m.satisfy(f1, g))
# how about if x is Chris?
g['x'] = 'c'
print(m.satisfy(f1, g))

This is what the assignment function is for.  It is a record of who/what we are pointing at,
and with which fingers.  (This is really designed to handle pronouns like *he*, *she*, *it*.
If you use those pronouns, it is assumed that something in the discourse is basically pointing
at the individual you mean.  Without some kind of pointing ("deixis") you won't be able to
interpret the referent of a pronoun.)

### Parsing sentences ###

Let's try to build a little grammar that can take sentences and interpret them.  What we
want to do here is create some phrase structure rules that will apply the semantics we
defined to a syntactic structure.  We'll build this up from the bottom.

As a first step, we will define the NPs, which will be just the names we have.
(We are going to build a big multi-line string and then create the grammar using a
`fromstring` function.)

In [ ]:
npdef = r"""
NP[SEM=<andrea>] -> 'andrea'
NP[SEM=<bobby>] -> 'bobby'
NP[SEM=<chris>] -> 'chris'
NP[SEM=<dana>] -> 'dana'
NP[SEM=<the_sun>] -> 'the_sun'
NP[SEM=<the_moon>] -> 'the_moon'
"""

What this means is that if the English word 'andrea' is encountered, that can be
interpreted as an NP with the SEM feature being `<andrea>`.  And likewise for the other
proper names.

> **NOTE** Before the `"""` there is an `r`.  This is actually somewhat important later, because it means that the `\` (backslash) character will be interpreted as a backslash.  If we didn't have then `r` ("raw"), then Python would do some magic that would wind up treating the `\` character as an "escape" character.

As for how the whole tree combines, it will start with `S` at the top, which is
formed from an `NP` and a `VP`, and the `VP` is formed from a `V` and an `NP`.
For now, that's all we'll do.

What we want is for the semantics of the VP to combine the semantics of the V
with the semantics of the NP.  So, if the V is "likes(x, y)", and the NP is "bobby",
then we want the VP to be "likes(x, bobby)", more or less.

In [ ]:
cfgdef = r"""
% start S
S[SEM=<?vp(?subj)>] -> NP[SEM=?subj] VP[SEM=?vp]
"""

The way to understand this is: The semantics of S is the function that
we get from the semantics of VP, applied to the argument that we get from
the semantics of the NP subject.  So, by saying `NP[SEM=?subj]` we are
naming the value of the NP's `SEM` feature (whatever it is) as `?subj`.
We name the value of the VP's `SEM` feature (whatever it is) as `?vp`.
We assume that `?vp` is a function that can take `?subj` as an argument.
And so, the `SEM` feature that we assign to S is whatever we get when
we apply the function `?vp` to the argument `?subj`.

> Note that the `% start S` is actually important (it is not a comment).  The grammar needs to start with that, this tells the parser what symbol is at the top of the tree.

We then do the same thing for the VP.  We assume that the V is going
to be a function that we can apply to the NP.

In [ ]:
cfgdef += r"""
VP[SEM=<?v(?obj)>] -> V[SEM=?v] NP[SEM=?obj]
V[SEM=<\y.\x.likes(x,y)>] -> 'likes'
"""

> Above was where the `r` was important, because we have some `\` characters in the string.  We need those `\` characters to stay there.  They are being treated as "lambda" characters.  We may not have discussed these yet.  For the moment, maybe just trusting the notation above is wisest.  What the second line says, though, is really: "given a y, and given an x, x likes y".  The `\` indicates that the expression takes an argument, and the fact that there are two means that we need two arguments (liker and likee) before we know whether the liking relationship truly holds.

So, now we can add in the NP definitions we did at the beginning, and
take a look at the whole grammar.

In [ ]:
cfgdef += npdef
print(cfgdef)

Now that we have the definition, we can parse it into an actual grammar
that NLTK can use, and then connect it to a parser (we will use the one
called `FeatureChartParser`).

In [ ]:
from nltk import grammar
gram = grammar.FeatureGrammar.fromstring(cfgdef)
cp = nltk.FeatureChartParser(gram)

And now we can parse some sentences.  Let's start with "bobby likes chris":

In [ ]:
parses = list(cp.parse('bobby likes chris'.split()))
print(len(parses))
print(parses[0])

If everything worked up to now, you should see that there is 1 parse,
and `print(parses[0])` will show you the parse it got.

The very first line is the overall semantic value for the tree, which we
can get like this:

In [ ]:
treesem = parses[0].label()['SEM']
print(treesem)

And, now that we have this expression, we can test it against the model
to see if it is actually true.  Note that we are using `satisfy` and not
`evaluate` -- the `evaluate` function takes a string and turns it into a
semantic expression, and then calls `satisfy`.  Since we already have a
semantic expression, we can just call `satisfy` directly.

In [ ]:
print(m.satisfy(treesem, g))

And thus we learn that, in this model, Bobby does not like Chris.

If we want to know if Bobby likes Dana, we just change the sentence.

In [ ]:
parses = list(cp.parse('bobby likes dana'.split()))
print(parses[0])
treesem = parses[0].label()['SEM']
print(treesem)
print(m.satisfy(treesem, g))

## TASK 7

Use this grammar to parse sentences telling you whether Chris likes Bobby and
whether Bobby likes the Sun.

> Don't forget that the Sun is all one word (`the_sun`) in this grammar.


In [ ]:
# Answer 7: Parse sentences and check against the model for:
# Chris likes Bobby, and for Bobby likes the Sun



That's actually pretty cool.  We can get from a sentence to a tree to truth conditions to
an actual evaluation of whether a sentence is true or false.  Granted, we can't do very
complicated sentences, but we have a place to start and we can kind of see how we could
proceed.

# Defining and Displaying SHRDLU's world

Now, we will shift gears.  We've done some semantic definitions, world design, and some basic parsing.  It's clear how this is going to be useful in building this SHRDLU robot.  The semantics will get more complicated next time.  But for now, let's leave the semantics where they are and turn to some of the basic framework questions about how to represent and draw the SHRDLU world.

The basic parts of the program are:

- representation of the objects in the world
- grammar for syntactic parsing and semantic composition
- display module to show the current state of the world
- user input loop
- interpretation of user input to respond

What we are going to focus on here will be the simpler aspects of representation and display.

## Setting up the world ##

The setup of the world is the same sort of thing that we did above.
We are creating a model of the world, with a domain
of individuals, and a valuation function that determines how
the objects are arranged in the world and what properties they
have.

Rather than having a 3D world, for simplicity and since it
doesn't really matter, our world just has 8 squares in a line
on which things can be piled.

So, let's begin.  You can start with this:

In [ ]:
squares = ['s1', 's2', 's3', 's4', 's5', 's6', 's7', 's8']
dom = {'a', 'b', 'c', 'd', 'e'} | set(squares)
valstr = """
square => {s1, s2, s3, s4, s5, s6, s7, s8}
odd => {s1, s3, s5, s7}
even => {s2, s4, s6, s8}
block => {a, b}
pyramid => {c, e}
table => {d}
thing => {a, b, c, d, e}
red => {a}
blue => {b, e}
green => {c, d}
on => {(a,s1),(b,s2),(d,s4),(c,d)}
"""
val = nltk.sem.Valuation.fromstring(valstr)
val['held'] = {('e',)}
m = nltk.Model(dom, val)
g = nltk.Assignment(dom)

This completely specifies the world now.  We have 13 objects, 8 of which are
squares that represent the floor (4 of which are odd, 4 of which are even),
and 5 of which are shapes of various kinds (block, pyramid, table) with various
properies (red, big, etc.).  Three of the objects are on the floor, one object
is on another one, and one is in the robot hand.

## Looking at the world ##

It's kind of nice to be able to visualize what's happening, so we next want to
set up a way to display the state of the world.  Rather than risk using graphics,
we will be satisfied with a text representation.

So, the plan is this: Each of the floor squares will represent a column, and
we will stack shapes up from the floor.  A little bit above the stacks will
be the robot hand and the shape that it is holding.

Let's think a little bit about how we will put the information about our
world onto the screen.  This is a problem to solve, even if it is not really
natural language specific.  We want to have a 2D display, with the squares
along the bottom, and the objects arranged above them.  At the top,
we'll have some kind of representation of the robot hand.

The most simplified version of the world we just set up might look like
this:

```python
<A

   A
OO T
########
```

Where `#` represents a square, `O` represents a block, `T` represents
a table, `A` represents a pyramid, and `<` represents the robot hand.
In this image, we can see what shape everything is, and how they are
stacked up.

So, let's first try to display the world like this.  As you know from
our discussions in class, we will do something more complex later, but
I think starting with this simple representation of the world may help
clarify what we're doing in the more complex version.

I'm going to try to build this up by prototyping functions as we go.
First, something that, when given a list of rows, will print them.

In [ ]:
def draw_simple_world(rows):
    print('\n'.join(rows))

And the rows that we want for our world in its initial state, for this
simple representation, would be:

In [ ]:
initial_rows = [
    '<A      ',
    '        ',
    '   A    ',
    'OO T    ',
    '########'
    ]

And indeed, if we give those rows to `draw_simple_world()` we get the
desired output.  So far, nothing very difficult or surprising has happened.

In [ ]:
draw_simple_world(initial_rows)

But, now: How do we make this reflect the actual state of the model,
whatever it is?  How do we define those rows in an algorithmic way?

The top two rows are somewhat easy, since those rows are just the hand and
then an empty row.  So we'll start by looking at how to draw the first row,
the hand and what it's holding.

How do we know what to put in the hand?  We have
defined the predicate "held" in our valuation function, so the object
in the hand will be the one that has the property of being "held".

In [ ]:
print(val['held'])

We see that we can get the object being held by asking for `val['held']`
but what we want is to get `e` back (not `{('e',)}`.
So: how to get `'e'` out of that?  Well, it's the first element of the first/only
element, except that the concept of "first element" doesn't make sense for
a set.  So, we need to convert this to a list, retrieve the first element
(which will be `('e',)`) and then retrieve the first element of that.
The following function will do just this, and if nothing is being held, it
will return `None` instead.

In [ ]:
def obj_in_hand():
    if len(val['held']) == 0:
        return None
    else:
        return list(val['held'])[0][0]

You can see that it works by trying this:

In [ ]:
obj_in_hand()

Now that we know what object is being held, what shape is it?  Should we
draw an `A` (pyramid) in the hand?  Or a `O` (block)?  Or what?
To figure this out, we need to figure out what properties this object
has (apart from being "held").

What are properties?  They are defined in our model.  Specifically, each
property is a list of 
objects that have the property.  For example, the objects that have
the property of being a pyramid are:

In [ ]:
val['pyramid']

We see that `e` is in there, `e` is a pyramid (and `e` is also held).
So we know that we want to draw a pyramid in the hand because the thing
that is being held has the property of being a pyramid.

We are more generally going to want to know the properties of an object,
not just whether it is a pyramid, but also whether it is blue, etc.  So,
generally what we want is, for a given object, to be able to get a list
of all the properties/predicates it is a member of.
What are the predicates? The following will print a full list of them:

In [ ]:
print([v for v in val])

We want to include only those predicates that have a given object in
them.  So, the properties of `e` would be:

In [ ]:
print([v for v in val if ('e',) in val[v]])

It's a pyramid, it's blue, it's a thing, and it's being held.
The more general form, for any object, then, is:

In [ ]:
def obj_properties(obj):
    return {v for v in val if (obj,) in val[v]}

You should find that `obj_properties('e')` gives the same result as
the one we just got by hand.

## TASK 8

What are the properties of object `a`?

In [ ]:
# Answer 8: What are the properties of object 'a'?



If we want to figure out what shape an object is, we need to find
out which of the "shape properties" it has.  There are four properties
in our model that determine the shape of something.  Let's make a set
of those.

In [ ]:
shape_properties = {'block', 'pyramid', 'table', 'square'}

The shape of an object is whichever of the shape properties it has.
Since `obj_properties()` returns a full set of properties, we can take
that set and intersect it with a set of possible shape options,
so that whatever is in both sets (that is, is a possible shape option and is
a property of the object) will be the shape.

In [ ]:
shapes_only = obj_properties('e') & shape_properties
print(shapes_only)

The result is a (singleton, we are presuming) set containing
the object's shape.  We can extract the shape from the singleton
set like this:  

In [ ]:
print(list(shapes_only)[0])

Having done this by hand, we can define a function to automate the
process of getting us the shape more generally:

In [ ]:
def obj_shape(obj):
    shape_properties = {'block', 'pyramid', 'table', 'square'}
    shape_property = obj_properties(obj) & shape_properties
    if len(shape_property) == 0:
        return None
    else:
        return list(shape_property)[0]

Try it to see if it works:

In [ ]:
obj_shape('e')

Almost there, let's define a function that will draw the shape
that corresponds to an object.

In [ ]:
def draw_simple_shape(obj):
    shape_map = {'square': '#', 'pyramid': 'A',
                'block': 'O', 'table': 'T', None: ' '}
    shape = obj_shape(obj)
    return shape_map[shape]

Make sure it does what you expect.  The following should draw an
`A` (a pyramid).

In [ ]:
print(draw_simple_shape('e'))

## TASK 9

Show that this will draw a block (`O`) for a block,
a table (`T`) for a table, and a square (`#`) for a square.

In [ ]:
# Answer 9. Show that draw_simple_shape() will draw a block, table, square



OK!
We can now draw the first two lines, the one with the hand and the blank
line.  Let's get to it!  We'll draw the first two lines first, and
just hard-code the main part of the world for the moment.  Because this is
just the first version of this function, I've called it `build_simple_rows_v1()`.

In [ ]:
def build_simple_rows_v1(m, g):
    held_shape = draw_simple_shape(obj_in_hand())
    top_row = '<' + held_shape
    lower_rows = [
        '   A    ',
        'OO T    ',
        '########']
    display_rows = [top_row, ''] + lower_rows
    return display_rows

It still draws our world.

In [ ]:
draw_simple_world(build_simple_rows_v1(m, g))

And if we change what is held, the thing in the hand changes.

In [ ]:
val['held'] = {('a',)}
draw_simple_world(build_simple_rows_v1(m, g))

Let's change it back, though.

In [ ]:
val['held'] = {('e',)}
draw_simple_world(build_simple_rows_v1(m, g))

Ok, now to the more important part.  How do we draw those bottom lines?
How do we derive from our model that the definition of `lower_rows`
should wind up like this (based on the initial setup)?

```
    lower_rows = [
        '   A    ',
        'OO T    ',
        '########']
```

For the first row, we need to know to draw three spaces, and then a pyramid.
The reason that there are three spaces is that the things stacked on squares
1, 2, and 3 are not tall enough to reach this row.  But the things stacked on
square 4 do reach this row, and what is in this row above square 4 is a pyramid.

So, it should be clear at this point that before we build these rows, we need
to figure out what is stacked on each square so that we know how tall the stacks
are (and what the objects are in the stacks).  And in order to know what is
stacked on all the squares, we need to first be able to determine what is stacked
on any individual single square.  So that's our new sub-task.

To figure out what is on a square (and then what is on that object, and
so on), we consult the predicate "on" in our valuation function.  This is
a relation between objects that tells us what is on what.

Let's define `whats_on(obj)` to tell us what has the property of being "on"
a given object:

In [ ]:
def whats_on(obj):
    # ask: what is on the current support?
    f = nltk.sem.Expression.fromstring("on(x,s)")
    g2 = nltk.Assignment(dom, [('s', obj)])
    try:
        next_obj = list(m.satisfiers(f, 'x', g2))[0]
    except:
        next_obj = None
    return next_obj

This is fairly straightforward.  We set up a formula that contains two variables
(`x` and `s`) and set an assignment to point to the `obj` we are checking with `s`.
Then we ask the model for a set of the `x`es that are `satisfiers` of `on(x,s)`.
(In prose, that is asking "what are the things that are on `s`?")
The reason that we put the check for satisfiers in a `try` block is that if there are
no individuals in the domain that satisfy the formula (that is, if nothing is on this
object), then it throws an error.  By using `try...except` we can check for the error
and, if there is an error, set `next_obj` to be `None`.  As for what's happening in the
successful case...

## TASK 10

Describe in words what is happening in the line between `try` and `except` above.  Maybe
more specifically,
why did I enclose `m.satsifiers(...)` in `list(...)[0]`?

**Answer 10** (markdown)

So, we now have a function `whats_on(obj)` that gives us back what object is on `obj` (or
`None` if nothing is on `obj`).  We're well on our way to constructing the stacks now,
we just need to see what's on a square, what's on that, what's on that, until we're at the
top of the stack, for each stack.

**ACTIVITY**. Try it out.  Use `whats_on()` to see what's on
square 1, square 3, square 4, and on 'd'.

In [ ]:
# ACTIVITY, use whats_on() to see what is on squares 1, 3, and 4, and on 'd'



Now, to construct a stack above a single square, we can start with the square,
add the object that is on it, and any on that, etc., until we have a list of objects
that represent the stack on a given square.

In [ ]:
def build_stack(square):
    stack = [square]
    while True:
        next_obj = whats_on(stack[-1])
        if next_obj:
            stack.append(next_obj)
        else:
            break
    return stack

Here, we start by putting the square at the bottom of the stack,
and we look for what's on the last object in the stack repeatedly until
there isn't anything more.  If `whats_on()` returns `None` it will `break` 
out of the `while True` loop and return the stack we have built.

**ACTIVITY**. Try it out, this should also work.  Just give `build_stack()` a
square and it should give you the stack on that square (including the square).
Square 4 is the only one that has more than one thing on it.

In [ ]:
# Activity: try giving build_stack some squares and see what is on them.



We have all the parts in place now to draw the simple world.  So now we can make
a more sophisticated version of `build_simple_rows()`, one that includes the
creation of the `lower_rows` part as well.

Let's do a couple of steps by hand first.  To begin, let us assemble the whole
set of stacks.

In [ ]:
stacks = [build_stack(s) for s in squares]
print(stacks)

Ok, and now comes the engineering.  Recall: what we want is to get from the contents of `stacks` above to a representation that looks like this:

```
['   A    ',
 'OO T    ',
 '########']
```

In particular, for that topmost row, we need three spaces (representing 
the fact that none of the first three squares have piles that reach up
that high), then the shape at the top of the stack above square four,
and then four more spaces (because again, nothing is piled up that high
over squares 5 to 8).

So, imagine that we want to assemble that first row.  Let's first skip
ahead to the stack over square four.  What do we want to draw in the first
line for square four's stack?  We look at `stacks[3]` and we see that it
is `['s4', 'd', 'c']`.  So we want to draw the shape that corresponds to
`'c'` because, it is the third element, the furthest up and the one that
intersects with the current row we are drawing.  It is, in particular, in
`stacks[3][2]`.

What do we want to draw next in that line, over square 5?  Well, if we look
at `stacks[4]` we see that it is only 1 element long (it is just `['s5']`).
And we are drawing the line that corresponds to the third element in a stack.
So we would draw `stacks[4][2]` except that `stacks[4]` has no third element.
One possibility then, when we are drawing this line is just to try to retrieve
the third element for each stack, draw it if we get one, and draw a space if
we fail (because there aren't three elements in the stack).  This is probably 
the most straightforward approach.

So, let's walk through this in order and assemble this.

Suppose you want to know what is in the second stack in the first row and then in the second row.  You can check these like this:

In [ ]:
# second stack, first row
print(stacks[0][0])
# second stack, second row
print(stacks[0][1])

In [ ]:
# second stack third row? Well, there is no third row.
print(len(stacks[0]))

So, if we want to build the third row, we can do something like this:

In [ ]:
# build the third row (index 2)
row = ''
for s in stacks:
    if 2 < len(s):
        row += draw_simple_shape(s[2])
    else:
        row += ' '
print("'{}'".format(row))

And it is pretty simple to extend this to draw each row up to the one corresponding to the tallest stack, and collect those in a list. 

In [ ]:
# What is the height of the tallest stack?
tallest = max([len(s) for s in stacks])
print(tallest)

In [ ]:
# build the rows up to row corresponding to the tallest stack
tallest = max([len(s) for s in stacks])
rows = []
for i in range(tallest):
    row = ''
    for s in stacks:
        if i < len(s):
            row += draw_simple_shape(s[i])
        else:
            row += ' '
    rows.append(row)
print(rows)

That's just what we want, except that the rows are reversed (because we built it up from the bottom).  So, now we can revise the `build_simple_rows_v1()` function from above by replacing the hard-coded `lower_rows` with the ones we compute.  We can use `reversed()` to get them in the right order.

In [ ]:
def build_simple_rows(m, g):
    held_shape = draw_simple_shape(obj_in_hand())
    top_row = '<' + held_shape
    stacks = [build_stack(s) for s in squares]
    tallest = max([len(s) for s in stacks])
    lower_rows = []
    for i in range(tallest):
        row = ''
        for s in stacks:
            if i < len(s):
                row += draw_simple_shape(s[i])
            else:
                row += ' '
        lower_rows.append(row)
    display_rows = [top_row, ''] + list(reversed(lower_rows))
    return display_rows

And if everything has been working correctly up to now, you can
again try to draw the world and it will appear
(but now actually based on the state of the model!).

In [ ]:
draw_simple_world(build_simple_rows(m,g))

## TASK 11

Modify the world so that pyramid 'c' is on 'a' instead (and not on 'd'), and then draw the world.
(This is a bit of a challenge, you've got to both have been following along and remember how we set up the world a while back.  You'll want to change one of the pairs of the predicate `on` in `val` to change the world, and draw with `draw_simple_world()`.)

> If you don't want to fully redefine `val['on']` but instead just incrementally change it, you can make use of `val['on'].remove(x)`, which will remove the element `x` from `val['on']` (so, you need to put something specific in for `x` there in order to make it so that 'c' is no longer on 'd').  Note that the argument of `remove` is an element, whereas the argument of `update` is a set.  Given that these both change the state of `val` it might just be simpler for debugging purposes to just redefine it, though.

In [ ]:
# Answer 11. Revise the world so that 'c' is on 'a' and draw it.



Perfect. Let's leave that for now, we have enough to draw the world
in at least a simple way.

And, we'll leave the project there.  To recap, we have worked out how to write a simple semantic parser, and we've worked out how to represent and draw the world.  Next time we will design a semantics for the block world and work out the interaction model so we can actually talk to the robot. 